# Инициализация графа

In [26]:
import os
from dotenv import load_dotenv
import networkx as nx
from collections import defaultdict
import numpy as np
from scipy.spatial import cKDTree
import math
# подгружаем .env
load_dotenv()

# Считываем все креды
src_dir = os.environ.get('DB_SOURCE_DIR')

# Считываем родительскую директорию
from pathlib import Path
current_dir = Path().resolve()
project_root = current_dir.parent

Считаем данные о топологии остатков

In [27]:
import json
import pandas as pd

with open(f"../data/raw/topology_complete.json", 'r', encoding='utf-8') as f:
    data_topology = json.load(f)

Считываем pdb файл со структурой белка

In [28]:
atoms_df = pd.read_csv(f"{project_root}/{src_dir}/pdbs/analysis_qm/analysis4/mut2_qm.pdb",
                       sep="\\s+",
                       header=None,
                       names=['type', 'atom_id', 'name', 'residue', 'chain', 'residue_id', 'x', 'y', 'z', 'par1', 'par2', 'element'])

atoms_df = atoms_df.drop(columns=['type', 'chain', 'par1', 'par2', 'element'])

Создадим новый объект словарь смежности для более удобного взаимодействия со связями из файла топологии. При этом помним, что если решим переводить снова в json файл, то необходимо будет преобразовать в обычный словарь.

In [29]:
for residues in data_topology.keys():
    adjacency = defaultdict(set)
    for u, v in data_topology[residues]['bonds']:
        adjacency[u].add(v)
        adjacency[v].add(u)

    #data_topology[residues]['adjancey_bonds'] = dict(adjacency)
    data_topology[residues]['adjancey_bonds'] = adjacency

Инициализируем ноды - атомы и ковалентные связи

In [30]:
# represent bonds in protein as edges, using topology file as instruction. Also add nodes as atoms in atoms graph and amino acids in residue graph
def create_edge(res_id):
    # add new node for atom grapha and add to it property: donor/acceptor/None
    def add_node(node_id):
        name = atoms_df.at[node_id-1, 'name']
        if name in donor_role:
            props = {'donor-acceptor': donor_role[name]}
        elif name in acceptor_role:
            props = {'donor-acceptor': acceptor_role[name]}
        else:
            props = {'donor-acceptor': 'nan'}
        G_atoms.add_node(node_id, **props)

    # DataFrame of atoms for target residue
    res_df = atoms_df[atoms_df['residue_id'] == res_id]
    # get residue name according to its residue id
    res_name = res_df['residue'].iat[0]
    # get topology informatiin for picked residue
    res_topology = data_topology[res_name]
    # get information of existing bonds in aminoacid/protein
    existing_bonds = res_topology['adjancey_bonds']
    # get info about atoms belongs to residue
    res_atoms = atoms_df.loc[atoms_df['residue_id'] == res_id, 'atom_id']
    
    donor_role = {}
    for pair in res_topology['donors']:
        donor_role[pair[0]] = 'donor'
        #donor_role[pair[1]] = 'pre-donor'

    acceptor_role = {}
    for pair in res_topology['acceptors']:
        acceptor_role[pair[0]] = 'acceptor'
        #acceptor_role[pair[1]] = 'pre-acceptor'
    
    # run arround all atoms in residue and check if it has bond with another one. Complexity is O(N(N-1)/2) ~ O(N^2). But N is near 10-15, so actuall runtime is quate fast
    # and all atoms near each other, so methods like kd-trees don; improve it as mush as needed
    for id_in_res, atom_id in enumerate(res_atoms):
        add_node(int(atom_id))
        for second_atom_id in res_atoms[id_in_res+1::]:
            if atoms_df['name'][second_atom_id-1] in existing_bonds.get(atoms_df['name'][atom_id-1], []):
                G_atoms.add_edge(int(second_atom_id), int(atom_id), kind='covalent')

    if '+N' in res_topology['adjancey_bonds'].get('C', []):
        # find this residue’s 'C' atom(s)
        c_atom = int(res_df.loc[res_df['name'] == 'C', 'atom_id'].iloc[0])
        # find next residue’s 'N' atom(s)
        next_res = atoms_df[atoms_df['residue_id'] == res_id+1]
        n_atom = next_res.loc[next_res['name'] == 'N', 'atom_id']
        if not n_atom.empty:
            n_atom = int(n_atom.iloc[0])
            # add nodes and edges
            add_node(c_atom)
            add_node(n_atom)
            G_atoms.add_edge(c_atom, n_atom, kind='covalent')
            G_residues.add_edge(res_id, res_id+1, kind='covalent')

In [31]:
# nodes is ids of atoms from pdb file
G_atoms = nx.Graph()
# nodes is ids of residues from pdb file
G_residues = nx.Graph()

# loop arround all residues in protein
for res_id in atoms_df['residue_id'].unique().tolist():
    create_edge(res_id)
    G_residues.add_node(res_id, kind='covalent')

Добляем водородные связи

In [43]:
coords = atoms_df[["x", "y", "z"]].to_numpy()
kd = cKDTree(coords)

# 3) Prepare donor & acceptor lists
# Map atom_id → index in DataFrame (0-based)
id_to_idx = {atom_id: idx for idx, atom_id in enumerate(atoms_df["atom_id"])}
# Identify donor & acceptor atom_ids
donors = [
    atom_id
    for atom_id, props in G_atoms.nodes(data=True)
    if props.get("donor-acceptor") == "donor"
]
acceptors = [
    atom_id
    for atom_id, props in G_atoms.nodes(data=True)
    if props.get("donor-acceptor") == "acceptor"
]

# 4) For each donor, find acceptors within 3.5Å and check angle >= 145°
for donor_id in donors:
    di = id_to_idx[donor_id]
    d_coord = coords[di]
    # query points within 2Å (radius)
    idxs = kd.query_ball_point(d_coord, r=3.5)
        
    for j in idxs:
        acceptor_id = atoms_df.iloc[j]["atom_id"]
        if acceptor_id not in acceptors:
            continue
        a_coord = coords[j]

        # Compute angle at donor between (donor→acceptor) and donor’s bond direction.
        # We take donor’s first bonded neighbor in G_atoms as reference:
        neighbors = list(G_atoms.neighbors(donor_id))
        if not neighbors:
            continue
        # Use the first neighbor to define bond direction vector
        ref_id = neighbors[0]
        ri = id_to_idx[ref_id]
        r_coord = coords[ri]

        # Vectors
        v1 = a_coord - d_coord   # donor→acceptor
        v2 = r_coord - d_coord   # donor→reference neighbor

        # Angle in degrees
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        theta = math.degrees(math.acos(np.clip(cos_theta, -1.0, 1.0)))

        if theta >= 150.0:
            # Add hydrogen‐bond edge
            G_atoms.add_edge(int(donor_id), int(acceptor_id), kind="hydrogen")
            G_residues.add_edge(int(atoms_df['residue_id'][donor_id-1]), int(atoms_df['residue_id'][acceptor_id-1]), kind="hydrogen")

Сохраняем полученные графы для будущего использования

In [33]:
os.mkdir(f'{project_root}/{src_dir}/graphs/analysis4/mut2_qm')
nx.write_graphml(G_atoms,    f"{project_root}/{src_dir}/graphs/analysis4/mut2_qm/atoms.graphml")
nx.write_graphml(G_residues,    f"{project_root}/{src_dir}/graphs/analysis4/mut2_qm/residues.graphml")

# Сравнение нескольких графов

Загружаем полученные графы

In [34]:
G1_atoms    = nx.read_graphml(f"{project_root}/{src_dir}/graphs/analysis4/mut1_qm/atoms.graphml")
G1_residues = nx.read_graphml(f"{project_root}/{src_dir}/graphs/analysis4/mut1_qm/residues.graphml")
G2_atoms    = nx.read_graphml(f"{project_root}/{src_dir}/graphs/analysis4/mut2_qm/atoms.graphml")
G2_residues = nx.read_graphml(f"{project_root}/{src_dir}/graphs/analysis4/mut2_qm/residues.graphml")

In [35]:
#debug
def hydrogen_edges_for_res_id(
    G_res: nx.Graph,
    residue: str):
    """
    Return all hydrogen-bond edges adjacent to a given residue.
    Each edge returned as (residue, neighbor).
    """
    edges = []
    for nbr in G_res.neighbors(residue):
        data = G_res[residue][nbr]
        if data.get('kind') == 'hydrogen':
            edges.append((residue, nbr))
    return edges
print(hydrogen_edges_for_res_id(G1_residues, '201'))
print(hydrogen_edges_for_res_id(G2_residues, '201'))

[('201', '199')]
[('201', '199')]


Считываем pdb файлы сравниваемых белков

In [36]:
atoms_df_1 = pd.read_csv(f"{project_root}/{src_dir}/pdbs/analysis_qm/analysis4/mut1_qm.pdb",
                       sep="\\s+",
                       header=None,
                       names=['type', 'atom_id', 'name', 'residue', 'chain', 'residue_id', 'x', 'y', 'z', 'par1', 'par2', 'element'])

atoms_df_1 = atoms_df_1.drop(columns=['type', 'chain', 'par1', 'par2', 'element'])

atoms_df_2 = pd.read_csv(f"{project_root}/{src_dir}/pdbs/analysis_qm/analysis4/mut2_qm.pdb",
                       sep="\\s+",
                       header=None,
                       names=['type', 'atom_id', 'name', 'residue', 'chain', 'residue_id', 'x', 'y', 'z', 'par1', 'par2', 'element'])

atoms_df_2 = atoms_df_2.drop(columns=['type', 'chain', 'par1', 'par2', 'element'])

Получим списки водородных связей

In [37]:
def hydrogen_edges(G):
    return {
        tuple(sorted((u, v)))
        for u, v, attrs in G.edges(data=True)
        if attrs.get("kind") == "hydrogen"
    }

h1 = hydrogen_edges(G1_atoms)
h2 = hydrogen_edges(G2_atoms)

Сопоставим два белка, сделая mapping индексов

In [38]:
merged = pd.merge(
    atoms_df_1, atoms_df_2,
    on = ['residue_id', 'residue', 'name'],
    how = 'outer',
    indicator = True,
    suffixes=('_old', '_new'),
)

Можно в дальнейшем проанализировать данный мэппинг, к примеру для работы с водой, точнее ее перенумерацией
```
matched_atoms = merged[merged['_merge'] == 'both']
deleted_atoms = merged[merged['_merge'] == 'left_only']
inserted_atoms = merged[merged['_merge'] == 'right_only']
```

In [39]:
h1_copy = h1.copy()
compared_hydrogens = pd.DataFrame()
conserved = []
gained = []
lost = []
for hydr in h1:
    wt_id_1 = int(hydr[0])
    wt_id_2 = int(hydr[1])
    
    wt_1_general_info = merged.loc[merged['atom_id_old'] == wt_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_wt_donor', 'residue': 'residue_wt_donor', 'residue_id': 'residue_id_wt_donor'}).iloc[0].to_dict()
    wt_2_general_info = merged.loc[merged['atom_id_old'] == wt_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_wt_acceptor', 'residue': 'residue_wt_acceptor', 'residue_id': 'residue_id_wt_acceptor'}).iloc[0].to_dict()
    
    mut_id_1 = merged.loc[merged['atom_id_old'] == wt_id_1, 'atom_id_new'].values[0]
    mut_id_2 = merged.loc[merged['atom_id_old'] == wt_id_2, 'atom_id_new'].values[0]
    # пока не разбираемся с отсутсвием присутсвием на уровне мутантов
    if np.isnan(mut_id_1) or np.isnan(mut_id_2):
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'only_wt'}
        #mut_info = {'name_mut_donor': None, 'residue_mut_donor': None, 'residue_id_mut_donor': None, 'name_mut_acceptor': None, 'residue_mut_acceptor': None, 'residue_id_mut_acceptor': None}
        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        #row.update(mut_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        continue
    elif (str(int(mut_id_1)), str(int(mut_id_2))) in h2:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'conserved'}
        mut_1_general_info = merged.loc[merged['atom_id_new'] == mut_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_donor', 'residue': 'residue_mut_donor', 'residue_id': 'residue_id_mut_donor'}).iloc[0].to_dict()
        mut_2_general_info = merged.loc[merged['atom_id_new'] == mut_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_acceptor', 'residue': 'residue_mut_acceptor', 'residue_id': 'residue_id_mut_acceptor'}).iloc[0].to_dict()

        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2.remove((str(int(mut_id_1)), str(int(mut_id_2))))
    else:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'lost'}
        #mut_info = {'name_mut_donor': None, 'residue_mut_donor': None, 'residue_id_mut_donor': None, 'name_mut_acceptor': None, 'residue_mut_acceptor': None, 'residue_id_mut_acceptor': None}
        row.update(wt_1_general_info)
        row.update(wt_2_general_info)
        #row.update(mut_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
    # remove bonds, that have already checked
    h1_copy.remove((str(int(wt_id_1)), str(int(wt_id_2))))

In [40]:
h2_copy = h2.copy()
for hydr in h2:
    mut_id_1 = int(hydr[0])
    mut_id_2 = int(hydr[1])
    mut_1_general_info = merged.loc[merged['atom_id_new'] == mut_id_1, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_donor', 'residue': 'residue_mut_donor', 'residue_id': 'residue_id_mut_donor'}).iloc[0].to_dict()
    mut_2_general_info = merged.loc[merged['atom_id_new'] == mut_id_2, ['name', 'residue', 'residue_id']].rename(columns={'name': 'name_mut_acceptor', 'residue': 'residue_mut_acceptor', 'residue_id': 'residue_id_mut_acceptor'}).iloc[0].to_dict()
        
    wt_id_1 = merged.loc[merged['atom_id_new'] == mut_id_1, 'atom_id_old'].values[0]
    wt_id_2 = merged.loc[merged['atom_id_new'] == mut_id_2, 'atom_id_old'].values[0]
    # пока не разбираемся с отсутсвием присутсвием
    if np.isnan(wt_id_1) or np.isnan(wt_id_2):
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'only_mut'}
        #wt_info = {'name_wt_donor': None, 'residue_wt_donor': None, 'residue_id_wt_donor': None, 'name_wt_acceptor': None, 'residue_wt_acceptor': None, 'residue_id_wt_acceptor': None}
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        #row.update(wt_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2_copy.remove((str(int(mut_id_1)), str(int(mut_id_2))))
    else:
        row = {'atom_id_old_donor': wt_id_1, 'atom_id_old_acceptor': wt_id_2, 'atom_id_new_donor': mut_id_1, 'atom_id_new_acceptor': mut_id_2, 'status': 'gain'}
        #wt_info = {'name_wt_donor': None, 'residue_wt_donor': None, 'residue_id_wt_donor': None, 'name_wt_acceptor': None, 'residue_wt_acceptor': None, 'residue_id_wt_acceptor': None}
        #row.update(wt_info)
        row.update(mut_1_general_info)
        row.update(mut_2_general_info)
        compared_hydrogens = pd.concat([compared_hydrogens, pd.DataFrame([row])], ignore_index=True)
        h2_copy.remove((str(int(mut_id_1)), str(int(mut_id_2))))

Сохраним полученный датафрейм в формате csv, убрав воду, т.к. надо разбираться в нумерации

In [41]:
compared_hydrogens = compared_hydrogens.convert_dtypes()
mask = (compared_hydrogens['residue_wt_donor'] == 'HOH') | (compared_hydrogens['residue_wt_acceptor'] == 'HOH') | (compared_hydrogens['residue_mut_donor'] == 'HOH') | (compared_hydrogens['residue_mut_acceptor'] == 'HOH')
rows_to_drop = compared_hydrogens[mask].index
compared_hydrogens = compared_hydrogens.drop(rows_to_drop)

In [42]:
compared_hydrogens.to_csv('compared_hydrogens.csv', index=False)